## Group3
### Members
1.Andrea Ulloa 
2.Ana Angulo (20171627)
3.Angela Coapaza (20171636) 

## An Inferencial Problem: The College-Educated Wage Gap
Using the data from the March Supplement of the U.S. Current Population Survey (2015) , in  this lab, we are going to focus in  payments of the college-educated workers and answer the following inference question:

What is the difference in predicted wages between workers with some college education (scl) versus college graduate workers(clg)?

To investigate the College-Educated Wage Gap, we consider the following log-linear regression model:

\begin{align}
\log(Y) &= \beta'X + \epsilon\\
&= \beta_1 SCL  + \beta_2' CLG + \beta_2'W   \epsilon,
\end{align}

Where  SCL is the indicator of workers with some college education( 1  if yes 0  otherwise), CLG is the indicator of college graduate workers (1  if yes 0  otherwise) and the  𝑊 's are controls explaining variation in wages. Considering transformed wages by the logarithm, we are analyzing the relative difference in the payment of workers with some college education and college graduate workers. 


## 1. Data Analysis

We consider the same subsample of the U.S. Current Population Survey (2015). Let us load the data set.

In [1]:
using Pkg

Pkg.add("DataFrames")
Pkg.add("Dates")
Pkg.add("CategoricalArrays")
Pkg.add("RData")

using RData
using DataFrames
using Dates
using Statistics  #upload data of R format 
using CategoricalArrays # categorical data 

    Updating registry at `C:\Users\ANA\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Manifest.toml`


In [4]:
rdata_read = load("../../data/wage2015_subsample_inference.RData")
data = rdata_read["data"]
names(data)
println("Number of Rows : ", size(data)[1],"\n","Number of Columns : ", size(data)[2],) #rows and columns

Number of Rows : 5150
Number of Columns : 20


***Variable description***

- occ : occupational classification
- ind : industry classification
- lwage : log hourly wage
- sex : gender (1 female) (0 male)
- shs : some high school
- hsg : High school graduated
- scl : Some College
- clg: College Graduate
- ad: Advanced Degree
- ne: Northeast
- mw: Midwest
- so: South
- we: West
- exp1: experience

In [6]:
describe(data)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,wage,23.4104,3.02198,19.2308,528.846,0,Float64
2,lwage,2.97079,1.10591,2.95651,6.2707,0,Float64
3,sex,0.444466,0.0,0.0,1.0,0,Float64
4,shs,0.023301,0.0,0.0,1.0,0,Float64
5,hsg,0.243883,0.0,0.0,1.0,0,Float64
6,scl,0.278058,0.0,0.0,1.0,0,Float64
7,clg,0.31767,0.0,0.0,1.0,0,Float64
8,ad,0.137087,0.0,0.0,1.0,0,Float64
9,mw,0.259612,0.0,0.0,1.0,0,Float64


To start our (causal) analysis, we compare the sample means given by college education

In [8]:
Z = select(data, ["lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1"])

data_scl = filter(row -> row.scl == 1, data)
Z_scl = select(data_scl,["lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1"] )

data_clg = filter(row -> row.clg == 1, data)
Z_clg = select(data_clg,["lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1"] )

means = DataFrame( variables = names(Z), All = describe(Z, :mean)[!,2], Some_College = describe(Z_scl,:mean)[!,2], College_Graduate = describe(Z_clg,:mean)[!,2])


,variables,All,Some_College,College_Graduate
,String,Float64,Float64,Float64
1,lwage,2.97079,2.84734,3.13367
2,sex,0.444466,0.453911,0.485941
3,shs,0.023301,0.0,0.0
4,hsg,0.243883,0.0,0.0
5,scl,0.278058,1.0,0.0
6,clg,0.31767,0.0,1.0
7,ad,0.137087,0.0,0.0
8,ne,0.227767,0.186453,0.261614
9,mw,0.259612,0.305866,0.231051


In particular, the table above shows that the difference in average logwage between men and women is equal to $0,28$

In [9]:
mean(Z_scl[:,:lwage]) - mean(Z_clg[:,:lwage])

-0.2863352099445877

Thus, the unconditional college education wage gap is about $28,6$\% for the group of college graduates.

This unconditional (predictive) effect of _____ equals the coefficient $\beta$ in the univariate ols regression of $Y$ on $D$:

\begin{align}
\log(Y) &=\beta D + \epsilon.
\end{align}

We verify this by running an ols regression in Julia.

In [13]:
#install all the package that we can need
Pkg.add("Lathe")
Pkg.add("GLM") # package to run models 
Pkg.add("StatsPlots")
Pkg.add("CSV")
Pkg.add("MLBase")
Pkg.add("Tables")
Pkg.add("CovarianceMatrices") # robust standar error 
# Load the installed packages
using DataFrames
using Tables
using Lathe
using CSV
using GLM
using CovarianceMatrices

   Resolving package versions...
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\ANA\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
   Installed WeakRefStrings ─ v1.4.2
   Installed FilePathsBase ── v0.9.18
   Installed CSV ──────────── v0.10.4
    Updating `C:\Users\ANA\.julia\environments\v1.7\Project.toml`
  [336ed68f] + CSV v0.10.4
    Updating `C:\Users\ANA\.julia\environments\v1.7\Manifest.toml`
  [336ed68f] + CSV v0.10.4
  [48062228] + FilePathsBase v0.9.18
  [ea10d353] + WeakRefStrings v1.4.2
Precompiling project...
  ✓ FilePathsBase
  ✓ WeakRefStrings
  ✓ CSV
  3 depe

In [15]:
nocontrol_model = lm(@formula(lwage ~ scl + clg), data)
nocontrol_est = GLM.coef(nocontrol_model)[2]
nocontrol_se = GLM.coeftable(nocontrol_model).cols[2][2]

nocontrol_se1 = stderror(HC1(), nocontrol_model)[2]
println("The estimated gender coefficient is ", nocontrol_est ," and the corresponding robust standard error is " ,nocontrol_se1)

The estimated gender coefficient is -0.08037054590503093 and the corresponding robust standard error is 0.019215703164420877
